In [1]:
%pylab qt5

Populating the interactive namespace from numpy and matplotlib


In [2]:
import scipy

In [3]:
from importlib import reload
import lib2
from IPython.display import clear_output
from lib import data_management as dm
from time import sleep
import drivers
from lib2.IQPulseSequence import *
from drivers.KeysightAWG import *
from drivers.IQAWG import *
warnings.filterwarnings("ignore",".*GUI is implemented.*")
mpl.rcParams['grid.color'] = 'k'
mpl.rcParams['grid.linestyle'] = ':'
mpl.rcParams['grid.linewidth'] = 0.5

In [218]:
if_freq = 100e6

calibrations_ro = (dm.load_IQMX_calibration_database("Xmons_Nb_bandage_res_mixer_5", -10))
ro_cal = calibrations_ro.get(frozenset(dict(lo_power=7, ssb_power=-35, lo_frequency=7.535e9, 
                                      if_frequency=0, waveform_resolution=1).items()))
ro_pb = PulseBuilder(ro_cal)

calibrations_q = (dm.load_IQMX_calibration_database("Xmons_Nb_bandage_qub_mixer_5", -6))
q_cal = calibrations_q.get(frozenset(dict(lo_power=8, ssb_power=-17, lo_frequency=6.163e9+if_freq, 
                                      if_frequency=if_freq, waveform_resolution=.5).items()))
q_pb = PulseBuilder(q_cal)

In [219]:
print(q_cal)

Calibration data for mixer Xmons_Nb_bandage_qub_mixer_5
Mixer parameters: {'mixer_id': 'Xmons_Nb_bandage_qub_mixer_5', 'iq_attenuation': -6}
Radiation parameters: {'lo_frequency': 6263000000.0, 'lo_power': 8, 'if_frequency': 100000000.0, 'ssb_power': -17, 'waveform_resolution': 0.5}
Optimization results: {'dc': -77.934364318847656, 'if': [-16.656984329223633, -46.59967041015625, -122.78775024414062]}
Optimization parameters {'dc_offsets': array([ 0.00379474, -0.016923  ]), 'dc_offsets_open': None, 'if_offsets': array([ 0.00398447, -0.02057001]), 'if_amplitudes': array([ 0.44026808,  0.46053226]), 'if_phase': array([-1.49267587])}
Optimization time: 0 h 2 m 34.77 s
Finished at: 2017-08-22 11:57:42.644058


In [10]:
ro_awg = KeysightAWG("AWG3")

In [11]:
q_awg = KeysightAWG("AWG2")

In [30]:
ro_iqawg = IQAWG(AWGChannel(ro_awg, 1), AWGChannel(ro_awg, 2))
q_iqawg = IQAWG(AWGChannel(q_awg, 1), AWGChannel(q_awg, 2))

In [94]:
from drivers.Keysight_DSOX2014 import *

In [95]:
dso = Keysight_DSOX2014("DSO2014")

In [ ]:
dso.set_time_range(1e-5)
dso.set_time_offset(1e-6)
dso.digitize()
times, channel_data = dso.get_data(*Channel.ALL)
axes = flatten(plt.subplots(4,1, sharex=True, figsize=(15,5))[1])
colors = ["C0", "C1", "C2", "C3"]
for idx, ax in enumerate(axes):
    ax.plot(times, channel_data[idx], color=colors[idx])
    ax.minorticks_on()
    ax.grid(which="both")
    ax.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
plt.tight_layout()

______________

### Resonators scan areas

In [163]:
scan_areas = [(7529945247.2623634, 7579945247.2623634),
  (7541330816.5408268, 7591330816.5408268),
  (7552773888.6944351, 7602773888.6944351),
  (7564561978.0989046, 7614561978.0989046),
  (7574164958.2479124, 7624164958.2479124)]

In [164]:
scan_areas

[(7529945247.262363, 7579945247.262363),
 (7541330816.540827, 7591330816.540827),
 (7552773888.694435, 7602773888.694435),
 (7564561978.098905, 7614561978.098905),
 (7574164958.247912, 7624164958.247912)]

# Spectroscopy

## Single-tone spectroscopy

In [8]:
#reload(lib2.SingleToneSpectroscopy)
from lib2.SingleToneSpectroscopy import *

In [254]:
ro_iqawg.output_pulse_sequence(ro_pb.add_dc_pulse(800,2).build())

In [165]:
res_limits = scan_areas[0]

In [255]:
STS = SingleToneSpectroscopy("IV-STS", "Xmons_Nb_bandage", vna = 'vna1', src = 'yok3')
vna_parameters = {"bandwidth":1000, "freq_limits":res_limits, "nop":2001, "sweep_type": "LIN", "power":-20, "averages":1}
STS.set_fixed_parameters(vna_parameters)
currents = linspace(0e-3, 2e-3, 51)
STS._src.set_current(currents[0])
sleep(1)
STS.set_swept_parameters({'current': (STS._src.set_current, currents)})
result = STS.launch()
STS._src.set_current(0);
result.visualize();

vna1 is already initialized
yok3 is already initialized
Started at:  2017-08-15 17:22:54.291988
Time left: 0 h 0 m 19.64 s, [current: 1.64e-03]: , average cycle time: 2.18 s       

In [257]:
result.visualize();

In [256]:
result.save();

In [300]:
STS._src.set_appropriate_range(0.01)

True

## Two-tone spectroscopy

In [13]:
# reload(lib2.Measurement)
# reload(lib2.TwoToneSpectroscopyBase)
# reload(lib2.TwoToneSpectroscopy)
from lib2.TwoToneSpectroscopy import *

In [14]:
q_iqawg.output_pulse_sequence(q_pb.add_dc_pulse(800,2).build())
ro_iqawg.output_pulse_sequence(ro_pb.add_dc_pulse(800,2).build())

In [15]:
ro_awg._visainstrument.write(":ARM:SOUR1 IMM")

(16, <StatusCode.success: 0>)

In [17]:
res_limits = (7525e6, 7545e6)

#### Qubit spectrum

In [23]:
TTS = FluxTwoToneSpectroscopy("I-two-tone", "Xmons_Nb_bandage", vna_name="vna1", 
                              mw_src_name="mxg", current_src_name="yok3")
vna_parameters = {"bandwidth":50, "freq_limits":res_limits, "nop":10, "sweep_type":"LIN", "power":-10, "averages":1}
mw_src_parameters = {"power":-15}
mw_src_frequencies = linspace(6.12e9, 6.20e9, 401)
currents = linspace(1.29e-3, 1.32e-3, 4)
TTS.setup_control_parameters(vna_parameters, mw_src_parameters, mw_src_frequencies, currents)
TTS._measurement_result.set_unwrap_phase(True)

vna1 is already initialized
mxg is already initialized
yok3 is already initialized
Detecting a resonator within provided frequency range of the VNA (7525000000.0, 7545000000.0)                    at qubit's sweet spot (1.30 mA)
Detected frequency is 7.53513 GHz, at 57.93 mU and 164.79 degrees


In [24]:
result = TTS.launch()

Started at:  2017-08-21 15:10:07.458586
Time left: 0 h 0 m 10.57 s, parameter value: 1.320e-03, average cycle time: 0.22 s          

In [20]:
result._unwrap_phase = True

In [22]:
result.visualize();

In [173]:
result.save()

________________

#### AC stark TTS from ro power

In [300]:
sum(avg_factors/100*160)/60

7.1803112671302305

In [301]:
exec(open("scripts/ac_stark_tts.py").read())

vna1 is already initialized
mxg is already initialized
yok3 is already initialized
Detecting a resonator within provided frequency range of the VNA (7770000000.0, 7790000000.0)                    at qubit's sweet spot (0.90 mA)
Detected frequency is 7.77972 GHz, at 575.29 mU and -43.27 degrees
Started at:  2017-08-15 21:59:32.717364
Time left: 0 h 0 m 0.0 s, parameter value: 1.150e-03, average cycle time: 0.47 s             vna1 is already initialized
mxg is already initialized
yok3 is already initialized
Detecting a resonator within provided frequency range of the VNA (7770000000.0, 7790000000.0)                    at qubit's sweet spot (0.90 mA)
Detected frequency is 7.77962 GHz, at 577.76 mU and -31.10 degrees
Started at:  2017-08-16 00:39:58.267914
Time left: 0 h 0 m 0.0 s, parameter value: 1.150e-03, average cycle time: 0.31 s             vna1 is already initialized
mxg is already initialized
yok3 is already initialized
Detecting a resonator within provided frequency range of the 

#### Linewidth from excitation power

In [283]:
ro_iqawg.output_pulse_sequence(ro_pb.add_dc_pulse(1000, 1).build())
q_iqawg.output_pulse_sequence(q_pb.add_dc_pulse(1000, 1).build())

In [22]:
PTS = PowerTwoToneSpectroscopy("VI-powerscan", "Xmons_Nb_bandage", vna_name='vna1', mw_src_name='mxg', current_src_name='yok3')
vna_parameters = {"bandwidth":200, "freq_limits":(7.52e9, 7.55e9), "nop":50, "power":7, "sweep_type":"CW","averages":1}
mw_src_parameters = {}
mw_src_frequencies = linspace(6.14e9, 6.18e9,401)
mw_src_powers = linspace(7, 7.1, 2)
current = 1.36e-3
PTS.setup_control_parameters(vna_parameters, mw_src_parameters, mw_src_frequencies, mw_src_powers, current)
PTS._measurement_result.set_phase_units("deg")

NameError: name 'PowerTwoToneSpectroscopy' is not defined

In [59]:
result = PTS.launch()

Started at:  2017-08-08 19:45:43.804415
Time left: 0 h 3 m 26.8 s, parameter value: 7.000e+00, average cycle time: 0.28 s           

In [51]:
result.visualize();

In [338]:
result.save()

#### AC Stark

In [ ]:
ro_awg.output_pulse_sequence(ro_pb.add_dc_pulse(100).build())
q_awg.output_pulse_sequence(q_pb.add_dc_pulse(100, 5).build())

In [15]:
ASTS = AcStarkTwoToneSpectroscopy("IV-ac-stark", "Xmons_Nb_bandage",
                                  vna_name='vna1', mw_src_name='mxg', current_src_name='yok3', )
vna_parameters = {"bandwidth":1000, "freq_limits":res_limits, "nop":2, "power":-25, "averages":400, "sweep_type":"LIN"}
mw_src_parameters = {"power":-5}
mw_src_frequencies = linspace(6.6e9, 7.4e9, 200)
vna_powers = linspace(-30, 0, 100)
current = 0.9e-3
ASTS.setup_control_parameters(vna_parameters, mw_src_parameters, mw_src_frequencies, vna_powers, current)

vna1 is already initialized
mxg is already initialized
yok3 is already initialized


In [16]:
result = ASTS.launch()

Started at:  2017-08-16 15:36:59.323302
Time left: 1 h 38 m 20.56 s, parameter value: -2.394e+01, average cycle time: 0.37 s                             

C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:427: RuntimeWarning: Number of calls to function has reached maxfev = 1400.
  warnings.warn(errors[info][0], RuntimeWarning)


Time left: 0 h 0 m 0.0 s, parameter value: 0.000e+00, average cycle time: 0.13 s            )                    

In [19]:
result.visualize()

(<matplotlib.figure.Figure at 0xe6ca898>,
 array([<matplotlib.axes._subplots.AxesSubplot object at 0x000000000E7120F0>,
        <matplotlib.axes._subplots.AxesSubplot object at 0x000000000E7E4198>], dtype=object),
 (<matplotlib.axes._axes.Axes at 0xe83f6a0>,
  <matplotlib.axes._axes.Axes at 0xe8fe780>))

In [18]:
result.save()

In [20]:
ASTS._mw_src.set_power(-15)

In [164]:
TTS.close_devs(["mxg"])

__________________________

# Time-domain measurements

### Check mixer calibration

In [44]:
q_lo = TTS._mw_src
q_lo.set_output_state("ON")
q_lo.set_frequency(6.054e9)
q_lo.set_power(10)

In [45]:
q_lo._visainstrument.close()

In [142]:
TTS._vna.set_nop(1)
TTS._vna.set_power(7)
TTS._vna.set_center(7.53511e9)
TTS._vna.sweep_continuous()

In [65]:
q_iqawg.output_pulse_sequence(q_pb.add_zero_pulse(20).build())
ro_iqawg.output_pulse_sequence(ro_pb.add_zero_pulse(20).build())

In [66]:
q_iqawg.output_pulse_sequence(q_pb.add_sine_pulse(20).build())
ro_iqawg.output_pulse_sequence(ro_pb.add_dc_pulse(20).build())

In [28]:
q_cal.get_radiation_parameters()

{'if_frequency': 50000000.0,
 'lo_frequency': 6210000000.0,
 'lo_power': 7,
 'ssb_power': -22,
 'waveform_resolution': 0.1}

### Set qubit frequency

In [25]:
TTS._current_src.set_current(1.31e-3)

True

In [23]:
q_freq

6164000000.0

In [26]:
q_freq = 6.163e9

In [140]:
q_freq = q_freq+1.98e6
q_freq

6164980000.0

### Rabi 

In [214]:
#reload(lib2.VNATimeResolvedDispersiveMeasurement)
#reload(lib2.VNATimeResolvedDispersiveMeasurement1D)
#reload(lib2.DispersiveRabiOscillations)
from lib2.DispersiveRabiOscillations import DispersiveRabiOscillations

In [220]:
ro_awg._visainstrument.write(":ARM:SOUR1 EXT")

(16, <StatusCode.success: 0>)

In [107]:
ro_iqawg.output_pulse_sequence(ro_pb.add_dc_pulse(1000).build())
#q_iqawg.output_pulse_sequence(q_pb.add_dc_pulse(1000).build())

In [119]:
DRO._pulse_sequence_parameters

{'excitation_amplitude': 1, 'modulating_window': 'rectangular'}

In [221]:
DRO = DispersiveRabiOscillations("I-rabi", "Xmons_Nb_bandage",
            vna_name="vna1", ro_awg=ro_iqawg, q_awg=q_iqawg, q_lo_name='mxg')
vna_parameters = {"bandwidth":50, "freq_limits":res_limits, "nop":4, "sweep_type":"LIN", "averages":4}
exc_frequency = q_freq
excitation_durations = linspace(0,800, 101)
rabi_sequence_parameters = {"awg_trigger_reaction_delay":390, "readout_duration":800, "repetition_period":10000}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DRO.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, exc_frequency, rabi_sequence_parameters)
DRO.set_swept_parameters(excitation_durations)

vna1 is already initialized
mxg is already initialized
Detecting a resonator within provided frequency range of the VNA (7525000000.0, 7545000000.0)                    
Detected frequency is 7.53515 GHz, at 3.55 mU and -176.92 degrees


In [222]:
result = DRO.launch()

Started at:  2017-08-22 11:58:17.815069
Time left: 0 h 0 m 0.0 s, [excitation_duration: 8.00e+02]: , average cycle time: 0.96 s         

In [210]:
result.visualize();

In [224]:
result.save()

In [223]:
pi_pulse_duration = result.get_pi_pulse_duration()*1e3
pi_pulse_duration

42.069313434702629

In [216]:
pi_pulse_duration = 58.66

### Decay

In [225]:
reload(lib2.VNATimeResolvedDispersiveMeasurement)
reload(lib2.VNATimeResolvedDispersiveMeasurement1D)
reload(lib2.DispersiveDecay)
from lib2.DispersiveDecay import *

In [226]:
DD = DispersiveDecay("II-decay", "Xmons_Nb_bandage",
            vna_name="vna1", ro_awg=ro_iqawg, q_awg=q_iqawg, q_lo_name='mxg', )
vna_parameters = {"bandwidth":20, "freq_limits":res_limits, "nop":40, "sweep_type":"LIN", "averages":5}
q_frequency = q_freq
readout_delays = linspace(0, 10000,101)
rabi_sequence_parameters = {"awg_trigger_reaction_delay":390, "readout_duration":500, 
                            "repetition_period":20000, "pi_pulse_duration":pi_pulse_duration}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DD.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, q_frequency, rabi_sequence_parameters)
DD.set_swept_parameters(readout_delays)

vna1 is already initialized
mxg is already initialized
Detecting a resonator within provided frequency range of the VNA (7525000000.0, 7545000000.0)                    
Detected frequency is 7.53505 GHz, at 0.80 mU and 167.87 degrees


In [ ]:
dd_result = DD.launch()

Started at:  2017-08-22 12:00:34.376880
Time left: 0 h 8 m 19.06 s, [readout_delay: 4.90e+03]: , average cycle time: 9.79 s        

In [125]:
dd_result.visualize();

In [126]:
dd_result.save()

### Rabi&Ramsey from readout_time

In [221]:
for ro_delay in ro_delays:
    print('\n Delay set'+str(ro_delay)+' ns;')
    # measure and save Rabi
    DRO = DispersiveRabiOscillations("I-rabi-with_"+str(ro_delay)+"_ns_readout", "Xmons_Nb_bandage",
            vna_name="vna1", ro_awg=ro_iqawg, q_awg=q_iqawg, q_lo_name='mxg')
    vna_parameters = {"bandwidth":20, "freq_limits":(7.525e9, 7.54501e9), "nop":20, "sweep_type":"LIN", "averages":1}
    exc_frequency = q_freq
    excitation_durations = linspace(0,2000, 201)
    rabi_sequence_parameters = {"awg_trigger_reaction_delay":390, "readout_duration":ro_delay, "repetition_period":8000}
    ro_awg_params =  {"calibration":ro_cal}
    q_awg_params = {"calibration":q_cal}
    DRO.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, exc_frequency, rabi_sequence_parameters)
    DRO.set_swept_parameters(excitation_durations)
    
    rabi_result = DRO.launch()
    
    rabi_result.save()
    
    pi_pulse_duration = rabi_result.get_pi_pulse_duration()*1e3
    # measure and save Ramsey
    DR = DispersiveRamsey("I-ramsey-with_"+str(ro_delay)+"_ns_readout", "Xmons_Nb_bandage", vna_name="vna1", ro_awg=ro_iqawg, 
                                                                                  q_awg=q_iqawg, q_lo_name='mxg')
    vna_parameters = {"bandwidth":10, "freq_limits":(7525e6, 7545.5e6), "nop":10, "averages":2, "sweep_type":"LIN"}
    ramsey_delays = linspace(0, 2000, 100)
    exc_frequency = q_freq+5e6
    rabi_sequence_parameters = {"awg_trigger_reaction_delay":390, "readout_duration":ro_delay, 
                            "repetition_period":10000, "half_pi_pulse_duration":pi_pulse_duration/2}
    ro_awg_params =  {"calibration":ro_cal}
    q_awg_params = {"calibration":q_cal}
    DR.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, exc_frequency, rabi_sequence_parameters)
    DR.set_swept_parameters(ramsey_delays)
    
    ramsey_result = DR.launch()
    
    ramsey_result.save()


 Delay set200.0 ns;
vna1 is already initialized
mxg is already initialized
Detecting a resonator within provided frequency range of the VNA (7525000000.0, 7545010000.0)                    
Detected frequency is 7.53506 GHz, at 3.88 mU and 171.94 degrees
Started at:  2017-08-12 18:30:53.802292
Time left: 0 h 0 m 0.0 s, [excitation_duration: 2.00e+03]: , average cycle time: 1.46 s         vna1 is already initialized
mxg is already initialized
Detecting a resonator within provided frequency range of the VNA (7525000000.0, 7545500000.0)                    
Detected frequency is 7.53510 GHz, at 1.08 mU and -179.22 degrees
Started at:  2017-08-12 18:35:54.379484
Time left: 0 h 3 m 47.48 s, [ramsey_delay: 8.08e+01]: , average cycle time: 2.39 s       

C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:779: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


Time left: 0 h 0 m 0.0 s, [ramsey_delay: 2.00e+03]: , average cycle time: 2.38 s         
 Delay set300.0 ns;
vna1 is already initialized
mxg is already initialized
Detecting a resonator within provided frequency range of the VNA (7525000000.0, 7545010000.0)                    
Detected frequency is 7.53506 GHz, at 0.87 mU and 174.43 degrees
Started at:  2017-08-12 18:39:55.660284
Time left: 0 h 0 m 0.0 s, [excitation_duration: 2.00e+03]: , average cycle time: 1.46 s         vna1 is already initialized
mxg is already initialized
Detecting a resonator within provided frequency range of the VNA (7525000000.0, 7545500000.0)                    
Detected frequency is 7.53510 GHz, at 1.80 mU and 175.63 degrees
Started at:  2017-08-12 18:44:56.256477
Time left: 0 h 0 m 0.0 s, [ramsey_delay: 2.00e+03]: , average cycle time: 2.4 s          
 Delay set400.0 ns;
vna1 is already initialized
mxg is already initialized
Detecting a resonator within provided frequency range of the VNA (7525000000.0, 7

### Ramsey

In [171]:
# reload(lib2.IQPulseSequence)
# reload(lib2.VNATimeResolvedDispersiveMeasurement)
# reload(lib2.VNATimeResolvedDispersiveMeasurement1D)
reload(lib2.DispersiveRamsey)
from lib2.DispersiveRamsey import *

In [172]:
DR = DispersiveRamsey("I-ramsey", "Xmons_Nb_bandage", vna_name="vna1", ro_awg=ro_iqawg, 
                                                                                  q_awg=q_iqawg, q_lo_name='mxg')
vna_parameters = {"bandwidth":10, "freq_limits":res_limits, "nop":1, "averages":2, "sweep_type":"LIN"}
ramsey_delays = linspace(0, 1000, 201)
exc_frequency = q_freq-10e6
rabi_sequence_parameters = {"awg_trigger_reaction_delay":390, "readout_duration":700, 
                            "repetition_period":10000, "half_pi_pulse_duration":pi_pulse_duration/2}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DR.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, exc_frequency, rabi_sequence_parameters)
DR.set_swept_parameters(ramsey_delays)

vna1 is already initialized
mxg is already initialized
Detecting a resonator within provided frequency range of the VNA (7525000000.0, 7545000000.0)                    
Detected frequency is 7.53505 GHz, at 3.45 mU and 169.09 degrees


In [173]:
dr_result = DR.launch()

Started at:  2017-08-21 18:53:33.508368
Time left: 0 h 2 m 27.07 s, [ramsey_delay: 2.50e+01]: , average cycle time: 0.75 s       

C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:779: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


Time left: 0 h 0 m 0.0 s, [ramsey_delay: 1.00e+03]: , average cycle time: 0.77 s         

In [139]:
dr_result.visualize();

In [135]:
dr_result.save()

### Echo

In [234]:
# reload(lib2.VNATimeResolvedMeasurement)
# from lib2.VNATimeResolvedMeasurement import *
# reload(lib2.DispersiveHahnEcho)
from lib2.DispersiveHahnEcho import *

In [245]:
DHN = DispersiveHahnEcho("III-hahn-echo", "Xmons_Nb_bandage",
            vna_name="vna1", ro_awg=ro_iqawg, q_awg=q_iqawg, q_lo_name='mxg')
vna_parameters = {"bandwidth":10, "freq_limits":res_limits, "nop":20, "sweep_type": "LIN", "averages":2}
q_frequency = q_freq
echo_delays = linspace(0, 7000, 101)
sequence_parameters = {"awg_trigger_reaction_delay":390, "readout_duration":500, 
                            "repetition_period":10000, "half_pi_pulse_duration":pi_pulse_duration/2}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DHN.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, q_frequency, sequence_parameters)
DHN.set_swept_parameters(echo_delays)

vna1 is already initialized
mxg is already initialized
Detecting a resonator within provided frequency range of the VNA (7685000000.0, 7700000000.0)                    
Detected frequency is 7.69352 GHz, at 4.17 mU and -135.07 degrees


In [246]:
dhn_result = DHN.launch()

Started at:  2017-08-15 15:45:53.871079
Time left: 0 h 0 m 0.0 s, [echo_delay: 5.00e+03]: , average cycle time: 5.06 s         

In [240]:
dhn_result.visualize();

In [247]:
dhn_result.save()

In [ ]:
MeasurementResult.load("Xmon Al BMSTU S444 2", "VI-hahn-echo").visualize()

### Ramsey fringes

In [150]:
# reload(lib2.Measurement)
# reload(drivers.KeysightAWG)
# reload(lib2.VNATimeResolvedDispersiveMeasurement)
# reload(lib2.VNATimeResolvedDispersiveMeasurement1D)
# reload(lib2.DispersiveRamseyFringes)
from lib2.DispersiveRamseyFringes import *

In [ ]:
q_freq = q_freq-5e6

In [149]:
DRF = DispersiveRamseyFringes("IV-ramsey-fringes", "Xmons_Nb_bandage",
                                      vna_name="vna1", ro_awg=ro_iqawg, q_awg=q_iqawg, q_lo_name='mxg')
vna_parameters = {"bandwidth":10, "freq_limits":res_limits, "nop":2, "sweep_type":"LIN", "averages":2}

excitation_freqs = linspace(q_freq-5e6, q_freq+5e6, 101)
ramsey_delays = linspace(10, 5000, 200)
sequence_parameters = {"awg_trigger_reaction_delay":0, "readout_duration":800, 
                            "repetition_period":20000, "half_pi_pulse_duration":pi_pulse_duration/2}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DRF.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, sequence_parameters)
DRF.set_swept_parameters(ramsey_delays, excitation_freqs)

vna1 is already initialized
mxg is already initialized
Detecting a resonator within provided frequency range of the VNA (7770000000.0, 7790000000.0)                    
Detected frequency is 7.77905 GHz, at 5.94 mU and 127.23 degrees


In [150]:
drf_result = DRF.launch()

Started at:  2017-08-18 08:56:13.936580
Time left: 2 h 22 m 32.85 s, [ramsey_delay: 6.02e+01, excitation_frequency: 7.30e+09]: , average cycle time: 0.43 s        

In [146]:
drf_result.visualize()

(<matplotlib.figure.Figure at 0xd251550>,
 array([<matplotlib.axes._subplots.AxesSubplot object at 0x000000000C492EB8>,
        <matplotlib.axes._subplots.AxesSubplot object at 0x0000000011A08278>], dtype=object),
  <matplotlib.axes._axes.Axes at 0x118355c0>])

In [232]:
drf_result.save()

### Rabi shevrons

In [136]:
# reload(lib2.VNATimeResolvedDispersiveMeasurement)
# reload(lib2.VNATimeResolvedDispersiveMeasurement2D)
#reload(lib2.DispersiveRabiChevrons)
from lib2.DispersiveRabiChevrons import *

In [147]:
DRC = DispersiveRabiChevrons("I-rabi-shevrons", "Xmons_Nb_bandage",
                                      vna_name="vna1", ro_awg=ro_iqawg, q_awg=q_iqawg, q_lo_name='mxg')
vna_parameters = {"bandwidth":10, "freq_limits":res_limits, "nop":4, "averages":2, "sweep_type": "LIN"}
q_frequency = 6.512e9
excitation_freqs = linspace(q_frequency-10e6, q_frequency+10e6, 51)
rabi_delays = linspace(0, 2000, 201)
sequence_parameters = {"awg_trigger_reaction_delay":390, "readout_duration":800, "repetition_period":10000}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DRC.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, sequence_parameters)
DRC.set_swept_parameters(rabi_delays, excitation_freqs)

vna1 is already initialized
mxg is already initialized
Detecting a resonator within provided frequency range of the VNA (7685000000.0, 7700000000.0)                    
Detected frequency is 7.69352 GHz, at 9.53 mU and -135.94 degrees


In [148]:
drc_result = DRC.launch()

Started at:  2017-08-15 10:01:47.133154
Time left: 0 h 0 m 0.0 s, [excitation_duration: 2.00e+03, excitation_frequency: 6.52e+09]: , average cycle time: 0.8 s           

In [218]:
drc_result.visualize();

In [219]:
drc_result.save()

### Dispersive shift measurement

In [61]:
#reload(lib2.VNATimeResolvedDispersiveMeasurement)
#reload(lib2.VNATimeResolvedDispersiveMeasurement2D)
#reload(lib2.TimeResolvedDispersiveShiftSpectroscopy)
from lib2.TimeResolvedDispersiveShiftSpectroscopy import *

In [71]:
res_limits = (7.605e9, 7.630e9)

In [74]:
DSS = TimeResolvedDispersiveShiftSpectroscopy("II-rabi-dispersive-shift-exact-q-freq", "Xmons_Nb_bandage",
                                      vna_name="vna1", ro_awg=ro_iqawg, q_awg=q_iqawg, q_lo_name='mxg')
vna_parameters = {"bandwidth":20, "freq_limits":res_limits, "nop":40, "sweep_type":"LIN","averages":1}
exc_frequency = q_freq
excitation_durations = linspace(0, 100, 101)
sequence_parameters = {"awg_trigger_reaction_delay":390, "readout_duration":250, "repetition_period":10000}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DSS.set_fixed_parameters(vna_parameters, exc_frequency, ro_awg_params, q_awg_params, sequence_parameters)
DSS.set_swept_parameters(excitation_durations)

vna1 is already initialized
mxg is already initialized


In [75]:
dss_result = DSS.launch()

Started at:  2017-08-14 18:23:51.160352
Time left: 0 h 0 m 0.0 s, [excitation_duration: 1.00e+02]: , average cycle time: 3.28 s         

In [70]:
dss_result.visualize();

In [69]:
dss_result.save()

In [ ]:
MeasurementResult.load("Xmon Al BMSTU S444 2", "VI-rabi-dispersive-shift").visualize();

______________________________________________

# Pulse optimization

## Amplified phase error

In [195]:
# reload(lib2.VNATimeResolvedDispersiveMeasurement)
# reload(lib2.VNATimeResolvedDispersiveMeasurement1D)
reload(lib2.DispersiveAPE)
from lib2.DispersiveAPE import *

### Rectangular pulses

In [204]:
for i in range(0, 11, 2):
    print(i)
    DAPE = DispersiveAPE("II-APE-%d-pseudo-I-pulses"%i, "Xmons_Nb_bandage", vna_name="vna1", ro_awg=ro_iqawg, 
                                                q_awg=q_iqawg, q_lo_name='mxg')
    vna_parameters = {"bandwidth":10, "freq_limits":res_limits, "nop":10**(1.7*i/10), "averages":3, "sweep_type": "LIN"}
    ramsey_angles = linspace(-4*pi, 4*pi, 100)
    exc_frequency = q_freq
    ro_awg_params =  {"calibration":ro_cal}
    q_awg_params = {"calibration":q_cal}
    APE_sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":500, 
                            "repetition_period":10000}
    APE_sequence_parameters.update({"half_pi_pulse_duration":pi_pulse_duration/2, 
                                    "pseudo_I_pulses_count":i, "modulating_window":'rectangular',
                                    "excitation_amplitude":1, "padding":10})
    DAPE.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, exc_frequency, APE_sequence_parameters)
    DAPE.set_swept_parameters(ramsey_angles)
    dape_result = DAPE.launch()
    dape_result.save()

0
vna1 is already initialized
mxg is already initialized
Detecting a resonator within provided frequency range of the VNA (7525000000.0, 7545000000.0)                    
Detected frequency is 7.53505 GHz, at 2.04 mU and 160.62 degrees
Started at:  2017-08-22 10:31:41.952883
Time left: 0 h 0 m 0.0 s, [ramsey_angle: 1.26e+01]: , average cycle time: 1.65 s          2
vna1 is already initialized
mxg is already initialized
Detecting a resonator within provided frequency range of the VNA (7525000000.0, 7545000000.0)                    
Detected frequency is 7.53505 GHz, at 2.13 mU and 164.37 degrees
Started at:  2017-08-22 10:34:34.873773
Time left: 0 h 0 m 0.0 s, [ramsey_angle: 1.26e+01]: , average cycle time: 1.93 s          4
vna1 is already initialized
mxg is already initialized
Detecting a resonator within provided frequency range of the VNA (7525000000.0, 7545000000.0)                    
Detected frequency is 7.53505 GHz, at 2.02 mU and 168.73 degrees
Started at:  2017-08-22 10:37:55

In [ ]:
dape_result = DAPE.launch()

In [ ]:
dape_result.visualize();

In [ ]:
dape_result.save()

### Gaussian pulses

#### Rabi

In [ ]:
# reload(lib2.VNATimeResolvedDispersiveMeasurement)
# reload(lib2.VNATimeResolvedDispersiveMeasurement1D)
# reload(lib2.DispersivePiPulseAmplitudeCalibration)
from lib2.DispersivePiPulseAmplitudeCalibration import *

In [ ]:
DPPAC = DispersivePiPulseAmplitudeCalibration("VI-pi-amp-cal", "Xmon Al BMSTU S444 2",
            vna_name="vna2", ro_awg_name="awg3", q_awg_name="awg2", q_lo_name='mxg')
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":10, "averages":2}
exc_frequency = q_freq
excitation_amps = linspace(0, 3, 20)
sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000,
                            "repetition_period":8000, "excitation_duration":50, "modulating_window":"gaussian"}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DPPAC.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, exc_frequency, sequence_parameters)
DPPAC.set_swept_parameters(excitation_amps)

In [ ]:
dppac_result = DPPAC.launch()

In [ ]:
dppac_result.visualize();

In [ ]:
pi_pulse_amplitude = 2.28
pi_pulse_amplitude

#### Ramsey 

In [ ]:
from lib2.DispersiveRamsey import *

In [119]:
DR = DispersiveRamsey("VI-ramsey", "Xmon Al BMSTU S444 2", vna_name="vna2", 
                                          ro_awg_name="awg3", q_awg_name="awg2", q_lo_name='mxg')
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":10, "averages":2}
ramsey_delays = linspace(0, 2000, 100)
exc_frequency = q_freq-5e6
ramsey_sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, 
                              "excitation_amplitude":pi_pulse_amplitude/2, "modulating_window":"gaussian",
                              "repetition_period":8000, "half_pi_pulse_duration":pi_pulse_duration/2}
DR.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, exc_frequency, ramsey_sequence_parameters)
DR.set_swept_parameters(ramsey_delays)

TypeError: __init__() got an unexpected keyword argument 'ro_awg_name'

In [ ]:
drg_result = DR.launch()

In [ ]:
drg_result.visualize();

#### APE

In [ ]:
for i in range(0, 11, 1):
    print(i)
    DAPE = DispersiveAPE("VI-APE-gaussian-%d-pseudo-I-pulses"%i, "Xmon Al BMSTU S444 2", vna_name="vna2", ro_awg_name="awg3", 
                                                q_awg_name="awg2", q_lo_name='mxg')
    vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":10**(1.7*i/10), "averages":3}
    ramsey_angles = linspace(-4*pi, 4*pi, 100)
    exc_frequency = q_freq
    ro_awg_params =  {"calibration":ro_cal}
    q_awg_params = {"calibration":q_cal}
    APE_sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, 
                            "repetition_period":8000, "padding":5,
                            "half_pi_pulse_duration":50, "modulating_window":"gaussian",
                            "excitation_amplitude":pi_pulse_amplitude/2, "pseudo_I_pulses_count":i}
    DAPE.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, exc_frequency, APE_sequence_parameters)
    DAPE.set_swept_parameters(ramsey_angles)
    dape_result = DAPE.launch()
    dape_result.save()
    clear_output()

In [ ]:
dape_result.visualize()

---------------------------

In [ ]:
reload(lib2.IQPulseSequence)
from lib2.IQPulseSequence import *

In [ ]:
pb = PulseBuilder(q_cal)

In [ ]:
APE_sequence_parameters["ramsey_angle"]=0
APE_sequence_parameters["pseudo_I_pulses_count"] = 1
APE_sequence_parameters["excitation_amplitude"] = 1

In [ ]:
APE_sequence_parameters

In [ ]:
PulseBuilder.build_dispersive_APE_sequences(pb, ro_pb, APE_sequence_parameters)[0].plot()
pb.add_zero_pulse(400).add_sine_pulse(65).add_sine_pulse(50).add_zero_pulse(15).build().plot()
plt.xlim(300,600)

In [ ]:
# pb.add_zero_pulse(10).add_sine_pulse(30).add_zero_pulse(10).build().plot()
pb.add_zero_pulse(10).add_sine_pulse(50, window=("gaussian", {"amplitude":10})).add_zero_pulse(10).build().plot()
plt.grid()

In [ ]:
rect = pb.add_zero_pulse(100).add_sine_pulse(300, amplitude=1).add_zero_pulse(1000).build()

In [ ]:
gaus = pb.add_zero_pulse(100).add_sine_pulse(300, amplitude=1, window="gaussian").add_zero_pulse(1000).build()

In [ ]:
rect.plot()
gaus.plot()

In [ ]:
q_awg.output_pulse_sequence(gaus)

In [ ]:
exp(-1/8)

In [ ]:
X = linspace(-1,1)

In [ ]:
plot(X, exp(-(3*X)**2/2**2))

In [ ]:
from resonator_tools import circuit

In [ ]:
circuit.notch_port()